## CBoW मॉडल का प्रशिक्षण

यह नोटबुक [AI for Beginners Curriculum](http://aka.ms/ai-beginners) का हिस्सा है।

इस उदाहरण में, हम CBoW भाषा मॉडल को प्रशिक्षित करने पर ध्यान देंगे ताकि हम अपना Word2Vec एम्बेडिंग स्पेस प्राप्त कर सकें। हम AG News डेटासेट को टेक्स्ट के स्रोत के रूप में उपयोग करेंगे।


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

पहले चलिए हमारा डेटासेट लोड करते हैं और टोकनाइज़र और शब्दावली को परिभाषित करते हैं। हम `vocab_size` को 5000 पर सेट करेंगे ताकि गणनाओं को थोड़ा सीमित किया जा सके।


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW मॉडल

CBoW $2N$ पड़ोसी शब्दों के आधार पर एक शब्द की भविष्यवाणी करना सीखता है। उदाहरण के लिए, जब $N=1$ हो, तो हमें वाक्य *I like to train networks* से निम्नलिखित जोड़े मिलेंगे: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks)। यहाँ, पहला शब्द पड़ोसी शब्द है जिसे इनपुट के रूप में उपयोग किया गया है, और दूसरा शब्द वह है जिसकी हम भविष्यवाणी कर रहे हैं।

अगले शब्द की भविष्यवाणी करने के लिए एक नेटवर्क बनाने के लिए, हमें पड़ोसी शब्द को इनपुट के रूप में देना होगा और शब्द संख्या को आउटपुट के रूप में प्राप्त करना होगा। CBoW नेटवर्क की संरचना निम्नलिखित है:

* इनपुट शब्द को एम्बेडिंग लेयर से गुजारा जाता है। यही एम्बेडिंग लेयर हमारा Word2Vec एम्बेडिंग होगा, इसलिए हम इसे अलग से `embedder` वेरिएबल के रूप में परिभाषित करेंगे। इस उदाहरण में हम एम्बेडिंग साइज = 30 का उपयोग करेंगे, हालांकि आप उच्च डाइमेंशन के साथ प्रयोग करना चाह सकते हैं (वास्तविक Word2Vec में 300 होता है)।
* एम्बेडिंग वेक्टर को फिर एक लीनियर लेयर में पास किया जाएगा, जो आउटपुट शब्द की भविष्यवाणी करेगा। इसलिए इसमें `vocab_size` न्यूरॉन्स होंगे।

आउटपुट के लिए, यदि हम `CrossEntropyLoss` को लॉस फंक्शन के रूप में उपयोग करते हैं, तो हमें अपेक्षित परिणाम के रूप में केवल शब्द संख्या प्रदान करनी होगी, बिना वन-हॉट एनकोडिंग के।


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## प्रशिक्षण डेटा तैयार करना

अब आइए मुख्य फ़ंक्शन को प्रोग्राम करते हैं, जो टेक्स्ट से CBoW शब्द जोड़े तैयार करेगा। यह फ़ंक्शन हमें विंडो साइज निर्दिष्ट करने की अनुमति देगा और इनपुट और आउटपुट शब्दों के जोड़े का एक सेट लौटाएगा। ध्यान दें कि इस फ़ंक्शन का उपयोग शब्दों पर किया जा सकता है, साथ ही वेक्टर/टेंसर पर भी - जो हमें टेक्स्ट को एन्कोड करने की अनुमति देगा, इससे पहले कि इसे `to_cbow` फ़ंक्शन में पास किया जाए।


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


आइए प्रशिक्षण डेटासेट तैयार करें। हम सभी समाचारों को देखेंगे, `to_cbow` को कॉल करेंगे ताकि शब्द जोड़ों की सूची प्राप्त हो सके, और उन जोड़ों को `X` और `Y` में जोड़ देंगे। समय बचाने के लिए, हम केवल पहले 10k समाचार आइटम पर विचार करेंगे - यदि आपके पास अधिक समय है और बेहतर एम्बेडिंग प्राप्त करना चाहते हैं, तो आप आसानी से इस सीमा को हटा सकते हैं :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

हम उस डेटा को एक डेटासेट में भी बदलेंगे, और डाटालोडर बनाएंगे:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

हम उस डेटा को एक डेटासेट में बदलेंगे, और डाटालोडर बनाएंगे:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

अब आइए वास्तविक प्रशिक्षण करें। हम `SGD` ऑप्टिमाइज़र का उपयोग करेंगे जिसमें काफी उच्च लर्निंग रेट होगा। आप अन्य ऑप्टिमाइज़र्स, जैसे `Adam`, के साथ भी प्रयोग कर सकते हैं। हम शुरुआत में 10 epochs के लिए प्रशिक्षण करेंगे - और यदि आप और भी कम हानि चाहते हैं तो आप इस सेल को पुनः चला सकते हैं।


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec आज़माना

Word2Vec का उपयोग करने के लिए, चलिए हमारे शब्दकोश में मौजूद सभी शब्दों के लिए वेक्टर निकालते हैं:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


यह वर्ड2वेक का उपयोग करके पर्यायवाची शब्दों को खोजने में दिलचस्प है। निम्नलिखित फ़ंक्शन दिए गए इनपुट के लिए `n` सबसे निकटतम शब्दों को लौटाएगा। उन्हें खोजने के लिए, हम $|w_i - v|$ का मान निकालते हैं, जहाँ $v$ हमारे इनपुट शब्द के लिए संबंधित वेक्टर है, और $w_i$ शब्दावली में $i$-वें शब्द का एनकोडिंग है। इसके बाद हम ऐरे को सॉर्ट करते हैं और `argsort` का उपयोग करके संबंधित इंडेक्स लौटाते हैं, और सूची के पहले `n` तत्व लेते हैं, जो शब्दावली में निकटतम शब्दों की स्थिति को एनकोड करते हैं।


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## मुख्य बातें

स्मार्ट तकनीकों जैसे CBoW का उपयोग करके, हम Word2Vec मॉडल को प्रशिक्षित कर सकते हैं। आप skip-gram मॉडल को प्रशिक्षित करने की कोशिश भी कर सकते हैं, जिसे केंद्रीय शब्द के आधार पर पड़ोसी शब्द की भविष्यवाणी करने के लिए प्रशिक्षित किया जाता है, और देख सकते हैं कि यह कितना अच्छा प्रदर्शन करता है।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
